## Prerequisites

In [26]:
%matplotlib inline

In [27]:
!pip install gym > /dev/null 2>&1

In [28]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [29]:
!pip install swig > /dev/null 2>&1
!pip install box2d-py > /dev/null 2>&1

In [30]:
!pip install Box2D > /dev/null 2>&1

In [31]:
!pip install gym[box2d] > /dev/null 2>&1

## Libraries

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import math, glob, io, base64, pickle, random

from collections import namedtuple, OrderedDict, deque
from itertools import count

import cv2

import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as T

import gym
from gym import spaces
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

from IPython.display import HTML
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display

## Utils

In [33]:
DEVICE= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
# Atari game screen resolution 1400x900
display = Display(visible=0, size=(1400, 900))
display.start()

In [35]:
def show_video():
  '''
  '''
  
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [36]:
def wrap_env(env):
  '''
  Wraps the environment for video rendering

  Input:
    env: environment instance - Instance of the environement in 
                                which the agent will act
  '''

  env = Monitor(env, './video', force=True)
  return env

In [ ]:
env = wrap_env(gym.make('SpaceInvaders-v0')) #wrapping the env to render as a video
env.reset()
env.render()
env.close()
show_video()

In [38]:
def convert_to_grayscale(image):
  '''
  '''
  
  return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [39]:
def frame_resize(image, resize_shape: tuple= (84, 84), method: any= 'crop', 
                 offset: int= 8):
  '''
  '''
  
  frame_h, frame_w= image.shape
  resize_h, resize_w= resize_shape

  if (method == 'crop') or (method == 1):
    frame_h= int(round(float(frame_h)*resize_w/frame_w))
    resized_frame= cv2.resize(image, (resize_w, frame_h), 
                              interpolation= cv2.INTER_LINEAR)
    cropped_frame= resized_frame[(frame_h - offset - resize_h):(frame_h - offset - resize_h + resize_h), :]
    return np.asarray(cropped_frame, dtype= np.uint8)

  elif (method == 'resize') or (method == 2):
    return np.asarray(cv2.resize(image, 
                                 resize_shape, 
                                 interpolation= cv2.INTER_LINEAR), 
                      dtype= np.uint8)
    
  else:
    raise Exception('Please choose one of the following methods: crop (1) or resize (2)')

## Environments

In [ ]:
class SimpleGrid(gym.Env):
    '''
    Source: https://github.com/sivashanmugamo/RL.git
    '''
    metadata= {'render.modes': []}

    def __init__(self, agent_position: list, goal_position: list, agent_value: any= 1, goal_value: any= 10, reward_set: dict= {}, grid_x: int= 3, grid_y: int= 3, stochasticity: bool= False, max_timesteps: int= 10) -> None:
        '''
        Initializes the properties of the environment
        Note: Only 4 actions are allowed in this environment - Left, Up, Right, & Down.
        Input:
            agent_position: list - Position of the Agent in the environment
            goal_postion: list - Position of the Goal in the environment
            agent_value: any - Value of the agent in the environment (Only for visualization | Won't add up for reward)
            goal_value: any - Value of the goal position
            reward_set: dict - Dictionary of reward's position (key) and the corresponsing reward (value)
            grid_x: int - Width of the grid environment
            grid_y: int - Height of the grid environment
            stochasticity: bool - If True, the environment is stochastic (uncertain), if false, deterministic (certain)
            max_timesteps: int - Maximum timesteps allowed per episode
        '''

        super().__init__()

        self.grid_x= grid_x
        self.grid_y= grid_y

        self.observation_space= spaces.Discrete(grid_x * grid_y)
        self.action_space= spaces.Discrete(4)

        self.init_agent_pos= agent_position.copy()
        self.agent_value= agent_value

        self.reward_set= reward_set

        self.init_goal_pos= goal_position.copy()
        self.goal_value= goal_value

        self.stochasticity= stochasticity
        self.max_timesteps= max_timesteps

        self.generate_state_lookup()
        self.reset()

    def generate_state_lookup(self) -> None:
        '''
        Generates a lookup table for the state (value) and its corresponding position (key)
        '''

        self.state_lookup= dict()

        state= 0
        for y in range(self.grid_y):
            for x in range(self.grid_x):
                self.state_lookup[(y, x)]= state
                state += 1

    def reset(self) -> int:
        '''
        Resets the environment to its default setup
        Returns:
            int: Agent's default state in the environment
        '''

        self.timestep= 0
        self.state= np.zeros((self.grid_y, self.grid_x))

        self.agent_pos= self.init_agent_pos
        self.state[tuple(self.agent_pos)]= self.agent_value

        if len(self.reward_set) > 0:
            for reward_pos, reward_val in self.reward_set.items():
                self.state[reward_pos]= reward_val

        self.goal_pos= self.init_goal_pos
        self.state[tuple(self.goal_pos)]= self.goal_value

        return self.state_lookup[tuple(self.agent_pos)]

    def save_environment(self, path) -> None:
        '''
        Saves environment parameters to a .pkl file
        Input:
            path: str - Path & filename 
        '''

        obj_to_save= dict()

        obj_to_save['grid_x']= self.grid_x
        obj_to_save['grid_y']= self.grid_y
        obj_to_save['observation_space']= self.observation_space
        obj_to_save['action_space']= self.action_space
        obj_to_save['init_agent_position']= self.init_agent_pos
        obj_to_save['agent_position']= self.agent_pos
        obj_to_save['agent_value']= self.agent_value
        obj_to_save['init_goal_position']= self.init_goal_pos
        obj_to_save['goal_position']= self.goal_pos
        obj_to_save['goal_value']= self.goal_value
        obj_to_save['reward_set']= self.reward_set
        obj_to_save['state']= self.state
        obj_to_save['stochasticity']= self.stochasticity
        obj_to_save['max_timesteps']= self.max_timesteps
        obj_to_save['timestep']= self.timestep

        pickle.dump(
            obj= obj_to_save, 
            file= open(path, 'wb')
        )

    def load_environment(self, path) -> None:
        '''
        Loads environment from a saved .pkl file
        Input:
            path: str - Path of the file
        '''

        with open(path, 'rb') as f:
            obj_to_load= pickle.load(f)
        
        self.grid_x= obj_to_load['grid_x']
        self.grid_y= obj_to_load['grid_y']
        self.observation_space= obj_to_load['observation_space']
        self.action_space= obj_to_load['action_space']
        self.init_agent_pos= obj_to_load['init_agent_position']
        self.agent_pos= obj_to_load['agent_position']
        self.agent_value= obj_to_load['agent_value']
        self.init_goal_pos= obj_to_load['init_goal_position']
        self.goal_pos= obj_to_load['goal_position']
        self.goal_value= obj_to_load['goal_value']
        self.reward_set= obj_to_load['reward_set']
        self.state= obj_to_load['state']
        self.stochasticity= obj_to_load['stochasticity']
        self.max_timesteps= obj_to_load['max_timesteps']
        self.timestep= obj_to_load['timestep']

        self.generate_state_lookup()

    def step(self, action) -> tuple:
        '''
        Performs the following,
            1. Moves the agent as per the chosen (given/random) action
            2. Sets the rewards per state in the environment
            3. Calcualtes the rewards as per action of the agent
        
        Input:
            action: int - Action to be performed from the action set
        Returns:
            state: int - Current state of the agent in the environment after action is performed (observation)
            reward: int - State reward
            done: bool - Denotes if the timestep is complete (or) goal is reached
            info: dict
        '''

        # Initializes the states
        self.state= np.zeros((self.grid_y, self.grid_x))
        temp_pos= self.agent_pos.copy()

        # Sets the uncertainity factor of the environment
        if self.stochasticity:
            action= action if random.uniform(0, 1) < 0.75 else random.choice([i for i in range(self.action_space.n)])

        # Moves the agent in the environment by 1 step
        if action == 0: # Down
            self.agent_pos[0] += 1
        if action == 1: # Up
            self.agent_pos[0] -= 1
        if action == 2: # Right
            self.agent_pos[1] += 1
        if action == 3: # Left
            self.agent_pos[1] -= 1

        # Keeps the agent within the confines of the environment
        if (self.agent_pos[0] < 0) or (self.agent_pos[1] > self.grid_y):
            self.agent_pos[0]= temp_pos[0]
        if (self.agent_pos[1] < 0) or (self.agent_pos[1] > self.grid_x):
            self.agent_pos[1]= temp_pos[1]

        self.state[tuple(self.agent_pos)]= self.agent_value

        # Sets the reward for the states
        if len(self.reward_set) > 0:
            for reward_pos, reward_val in self.reward_set.items():
                self.state[reward_pos]= reward_val
        
        self.goal_pos= self.init_goal_pos
        self.state[tuple(self.goal_pos)]= self.goal_value

        # Calculates the reward for the state & its corresponding action
        reward= 0
        if len(self.reward_set) > 0:
            for reward_pos, reward_val in self.reward_set.items():
                if (tuple(self.agent_pos) == reward_pos):
                    reward += reward_val

        if (self.agent_pos == self.goal_pos):
            reward+= self.goal_value

        # Time step increment
        self.timestep += 1

        done= True if ((self.timestep >= self.max_timesteps) or (self.agent_pos == self.goal_pos)) else False
        info= {}

        return (self.state_lookup[tuple(self.agent_pos)], reward, done, info)

    def render(self) -> None:
        '''
        Renders the current state of the environment
        '''
        
        plt.imsave('./environment_render.png', self.state)

    @property
    # Getter method for init_agent_pos
    def init_agent_pos(self):
        return self._init_agent_pos

    @init_agent_pos.setter
    # Setter method for init_agent_pos
    def init_agent_pos(self, value):
        # Restricts the value & type of init_agent_pos
        if isinstance(value, list) == False:
            raise TypeError('List expected for Agent Position, but received {}.'.format(type(value)))
        if (value[0] > self.grid_y-1) or (value[1] > self.grid_x-1):
            raise ValueError('Agent position should be between (0, 0) and ({}, {}).'.format(self.grid_x, self.grid_y))
        self._init_agent_pos= value

    @property
    # Getter method for init_goal_pos
    def init_goal_pos(self):
        return self._init_goal_pos

    @init_goal_pos.setter
    # Setter method for init_goal_pos
    def init_goal_pos(self, value):
        # Restricts the value & type of init_goal_pos
        if isinstance(value, list) == False:
            raise TypeError('List expected for Goal Position, but received {}.'.format(type(value)))
        if (value[0] > self.grid_y-1) or (value[1] > self.grid_x-1):
            raise ValueError('Goal position should be between (0, 0) and ({}, {}).'.format(self.grid_x, self.grid_y))
        self._init_goal_pos= value

    @property
    # Getter method for reward_set
    def reward_set(self):
        return self._reward_set

    @reward_set.setter
    # Setter method for reward_set
    def reward_set(self, value):
        # Restricts the value & type of reward_set
        if isinstance(value, dict) == False:
            raise TypeError('Dictionary expected for Reward Set, but received {}.'.format(type(value)))
        for key, val in value.items():
            if (key[0] > self.grid_y-1) or (key[1] > self.grid_x-1):
                raise ValueError('Reward position should be between (0, 0) and ({}, {}).'.format(self.grid_x, self.grid_y))
            if isinstance(val, (int, float, complex)) == False:
                raise ValueError('Reward should be numerical, but received {}.'.format(type(val)))
        self._reward_set= value

    @property
    # Getter method for max_timesteps
    def max_timesteps(self):
        return self._max_timesteps

    @max_timesteps.setter
    # Setter method for max_timesteps
    def max_timesteps(self, value):
        # Restricts the value & type of max_timesteps
        if isinstance(value, int) == False:
            raise TypeError('Int expected for maximum timesteps, but received {}.'.format(type(value)))
        if (value < 0):
            raise ValueError('Maximum timestep should be positive')

## DQN

In [17]:
class DQN(nn.Module):
  def __init__(self, env , input_dim: tuple, output_dim: int, 
               net_type: str= 'cnn', device: str= ''):
    '''
    '''
    super(DQN, self).__init__()

    self.device= device

    self.actions= np.arange(env.action_space.n)
    
    self.net_type= net_type
    self.input_dim= input_dim
    self.output_dim= output_dim

    self.build_cnn(input_dim= self.input_dim)

  def build_cnn(self, input_dim: tuple):
    '''
    '''
    self.conv1= nn.Conv2d(4, 32, 8, stride= 4, padding= 1)
    self.conv2= nn.Conv2d(32, 64, 4, stride= 2)
    self.conv3= nn.Conv2d(64, 128, 3)
    self.fc1= nn.Linear(128*19*8, 512)
    self.fc2= nn.Linear(512, 4)

    

In [18]:
transitions= namedtuple(
    typename= 'Transition', 
    field_names= ('state', 'action', 'next_state', 'reward'), 
    rename= False
)

class Experience(object):
  def __init__(self, capacity: int= int(1e5)):
    '''
    '''
    self.memory_buffer= list()
    self.capacity= capacity
    self.memory_count= 0

  def memorize(self, *observation: tuple):
    '''
    Stores the transition data - state, action, next_state, reward - in memory
    
    Input:
      observation: tuple - Tuple of state, action, next_state, reward
    '''

    if len(self.memory_buffer) < self.capacity:
      self.memory_buffer.append(observation)
    self.memory_buffer[self.memory_count]= transitions(*observation)

    # Rewrites old memory with new memory
    self.memory_count= (self.memory_count + 1) % self.capacity

  def sample(self, batch_size):
    '''
    Samples k number of observations from memory at random

    Input:
      batch_size: int - Number of observations per batch
    '''

    return random.sample(
        population= self.memory_buffer, 
        k= batch_size
    )

  def __len__(self):
    '''
    Returns the length of memory buffer
    '''

    return len(self.memory_buffer)

In [19]:
class Agent:
  def __init__(self, env, learning_rate: float= 0.01, gamma: float= 0.99, epsilon: float= 1.0, epsilon_decay: float= 0.1, discount_rate: float= 0.99, batch_size: int= 50, memory: int= int(1e5)):
    '''
    '''
    
    self.env= env
    self.lr= learning_rate
    self.gamma= gamma
    self.epsilon= epsilon
    self.epsilon_decay= epsilon_decay
    self.discount_rate= discount_rate
    self.batch_size= batch_size

    self.memory= Experience(capacity= memory)

    self.agent_net= DQN(input_dim= (84, 84, 4), output_dim= 4, net_type= 'cnn')

  def action(self, state):
    '''
    '''
    actions= self.agent_net.forward(state)
    if np.random.random() < 1-self.epsilon:
      action= torch.argmax(actions[1]).item()
    else:
      action= np.random.choice(self.env.action_space.n)
    self.steps += 1
    return action

  def learn(self):
    '''
    '''
    pass

SyntaxError: ignored

## Global Initializations

In [ ]:
# OpenAI Gym environments
ENV_1= 'CartPole-v0'
ENV_2= 'Breakout-v0'

## Environment run

In Atari environment, a set of consecutive frames observed at an instance by the player is a state.

By providing just the state to the neural network (NN) & let it calculate Q-value for each action, the computation becomes easier (as only one forward pass is necessary) and the Q-value corresponding to the action can be chosen from the output nodes.

## References
* OpenAI Gym Environment [Breakout] - https://gym.openai.com/envs/Breakout-v0/
* Deep Reinforcement Learning. Introduction. Deep Q Network (DQN) algorithm. [Medium] - https://medium.com/@markus.x.buchholz/deep-reinforcement-learning-introduction-deep-q-network-dqn-algorithm-fb74bf4d6862
* Rendering OpenAi Gym in Google Colaboratory - https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/
* Rendering OpenAi Gym in Google Colaboratory [Colab Notebook] - https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#scrollTo=8nj5sjsk15IT